# 1

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!ls gdrive/MyDrive/datasets/ham10000

HAM10000_images_part_1	HAM10000_images_part_2	HAM10000_metadata.csv


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(0)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

In [4]:
import torch
import torchvision.datasets
from torchvision import transforms

In [5]:
X_train = torch.load('gdrive/MyDrive/datasets/ham_100_X_train.pt')
y_train = torch.load('gdrive/MyDrive/datasets/ham_100_y_train.pt')
X_test = torch.load('gdrive/MyDrive/datasets/ham_100_X_test.pt')
y_test = torch.load('gdrive/MyDrive/datasets/ham_100_y_test.pt')

In [6]:
X_train /= 255.
X_test /= 255.

In [ ]:
X_train = torch.nn.functional.normalize(X_train)
X_test = torch.nn.functional.normalize(X_test)

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [7]:
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [8]:
import random

import torchvision.datasets
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [9]:
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks')

In [10]:
import pytorch_optimizer as optim

In [48]:
def train(net, X_train, y_train, X_test, y_test):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=5.0e-2)
    #optimizer = torch.optim.Adam(net.parameters(), lr=0.5e-3)
    optimizer = optim.PNMBelief(net.parameters(), lr=0.001)
    #optimizer = optim.YogiPNM(net.parameters())
    #optimizer = optim.AdaBelief(net.parameters())
    #optimizer = optim.DiffGrad(net.parameters(), lr=0.5e-3)
    #optimizer = optim.Yogi(net.parameters())
    
    batch_size = 10

    test_accuracy_history = []
    test_loss_history = []

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    for epoch in range(14):
        print("epoch ", epoch)
        order = np.random.permutation(len(X_train))
        
        if epoch == 9:
            #optimizer = torch.optim.SGD(net.parameters(), lr=5.0e-4)
            #optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-5)
            optimizer = optim.PNMBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.YogiPNM(net.parameters(), lr=1.0e-4)
            #optimizer = optim.AdaBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.DiffGrad(net.parameters(), lr=1.0e-5)
            #optimizer = optim.Yogi(net.parameters(), lr=1.0e-4)
    

        for start_index in tqdm(range(0, len(X_train), batch_size)):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = net.forward(X_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            
            X_batch

        net.eval()
        with torch.no_grad():
            test_preds = net.forward(X_test)
            loss_value = loss(test_preds, y_test).item()
            test_loss_history.append(loss_value)

            accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().item()
            test_accuracy_history.append(accuracy)
            #print(test_preds.argmax(dim=1))
            #print(y_test)

        print("accuracy = ", accuracy)
        print("\n\n")

    return test_accuracy_history, test_loss_history

In [12]:
accuracies = {}
losses = {}

In [13]:
from torchvision.models import resnext50_32x4d, ResNeXt50_32X4D_Weights

In [14]:
model = resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)

accuracies['resnext+SGD'], losses['resnext+SGD'] = \
    train(model, X_train, y_train, X_test, y_test)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 363MB/s]


epoch  0


100%|██████████| 802/802 [00:53<00:00, 15.09it/s]


accuracy =  0.6919620633125305



epoch  1


100%|██████████| 802/802 [00:46<00:00, 17.33it/s]


...

accuracy =  0.800798773765564



epoch  12


100%|██████████| 802/802 [00:46<00:00, 17.33it/s]


accuracy =  0.7978032827377319



epoch  13


100%|██████████| 802/802 [00:46<00:00, 17.34it/s]


accuracy =  0.8057913184165955





In [26]:
model = resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)

accuracies['resnext+Adam'], losses['resnext+Adam'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:52<00:00, 15.22it/s]


accuracy =  0.7613579630851746



epoch  1


100%|██████████| 802/802 [00:53<00:00, 14.91it/s]


accuracy =  0.7468796968460083



epoch  2


...

accuracy =  0.8072890639305115



epoch  12


100%|██████████| 802/802 [00:51<00:00, 15.43it/s]


accuracy =  0.8067898154258728



epoch  13


100%|██████████| 802/802 [00:52<00:00, 15.38it/s]


accuracy =  0.8117823004722595





In [34]:
model = resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
accuracies['resnext+Yogi'], losses['resnext+Yogi'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:55<00:00, 14.39it/s]


accuracy =  0.7493759393692017



epoch  1


100%|██████████| 802/802 [00:55<00:00, 14.36it/s]


accuracy =  0.7343984246253967



epoch  2


...

accuracy =  0.8252621293067932



epoch  12


100%|██████████| 802/802 [00:56<00:00, 14.31it/s]


accuracy =  0.8312531113624573



epoch  13


100%|██████████| 802/802 [00:55<00:00, 14.40it/s]


accuracy =  0.8342486023902893





In [42]:
model = resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
accuracies['resnext+DiffGrad'], losses['resnext+DiffGrad'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [01:07<00:00, 11.84it/s]


accuracy =  0.7558662295341492



epoch  1


100%|██████████| 802/802 [01:06<00:00, 12.09it/s]


accuracy =  0.7823265194892883



epoch  2


...

accuracy =  0.8312531113624573



epoch  12


100%|██████████| 802/802 [01:06<00:00, 12.11it/s]


accuracy =  0.8352471590042114



epoch  13


100%|██████████| 802/802 [01:06<00:00, 12.07it/s]


accuracy =  0.8312531113624573





In [49]:
model = resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
accuracies['resnext+PNMBelief'], losses['resnext+PNMBelief'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [01:03<00:00, 12.71it/s]


accuracy =  0.7463804483413696



epoch  1


100%|██████████| 802/802 [01:04<00:00, 12.41it/s]


accuracy =  0.779830276966095



epoch  2


...

accuracy =  0.8502246737480164



epoch  12


100%|██████████| 802/802 [01:03<00:00, 12.69it/s]


accuracy =  0.8452321290969849



epoch  13


100%|██████████| 802/802 [01:02<00:00, 12.75it/s]


accuracy =  0.850723922252655





# ansamble

In [15]:
APreds = pd.DataFrame()

In [50]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [17]:
APreds['SGD'] = test_preds.argmax(dim=1).tolist()

In [28]:
APreds['Adam'] = test_preds.argmax(dim=1).tolist()

In [36]:
APreds['Yogi'] = test_preds.argmax(dim=1).tolist()

In [44]:
APreds['DiffGrad'] = test_preds.argmax(dim=1).tolist()

In [51]:
APreds['PNMBelief'] = test_preds.argmax(dim=1).tolist()

In [52]:
APreds

,SGD,Adam,Yogi,DiffGrad,PNMBelief
0,4,4,4,4,4
1,4,4,4,4,4
2,4,4,4,4,4
3,2,4,4,4,4
4,1,4,4,4,4
...,...,...,...,...,...
1998,4,4,4,4,4
1999,4,5,4,5,4
2000,4,4,4,4,4
2001,4,4,4,4,4


In [ ]:
APreds = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_resnext.csv')
APreds = APreds.drop(['Unnamed: 0'], axis=1)

In [ ]:
APreds.to_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_resnext.csv')

In [60]:
ARes = pd.DataFrame(np.zeros((2003,7), dtype=int))

for i in range(2003):
    for j in range(5):
        if j == 0:
            ARes.iloc[i, APreds.iloc[i, j]] += 0 #0.80579
        if j == 1:
            ARes.iloc[i, APreds.iloc[i, j]] += 0 #0.81178
        if j == 2:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.83424
        if j == 3:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.83125
        if j == 4:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.85072

In [61]:
(torch.tensor(ARes.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8512231707572937

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(test_preds.argmax(dim=1).tolist(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #Adam

In [ ]:
D #PNMBelief

In [ ]:
D #Yogi

In [ ]:
D #DiffGrad

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(ARes.idxmax(axis=1).to_list(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #ansamble

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
precision_score(y_test, test_preds.argmax(dim=1), average='micro')


In [ ]:
print('precision: ', precision_score(y_test, test_preds.argmax(dim=1), average=None))
print('recall: ', recall_score(y_test, test_preds.argmax(dim=1), average=None))
print('f1_score: ', f1_score(y_test, test_preds.argmax(dim=1), average=None))

# SoftMax

In [19]:
ARes2 = pd.DataFrame(np.zeros((2003,7), dtype=int))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [20]:
m = torch.nn.Softmax(dim=1)

In [21]:
test_preds = m(test_preds) * 0.80579 #SGD

In [30]:
test_preds = m(test_preds) * 0.81178 #Adam

In [38]:
test_preds = m(test_preds) * 0.83424 #Yogi

In [46]:
test_preds = m(test_preds) * 0.83125 #DiffGrad

In [55]:
test_preds = m(test_preds) * 0.85072 #PNMBelief

In [56]:
ARes2 += pd.DataFrame(test_preds.to('cpu'))

In [ ]:
test_preds

In [40]:
ARes2

,0,1,2,3,4,5,6
0,0.000002,0.000003,0.000026,9.122376e-07,2.451754,0.000008,0.000016
1,0.000101,0.000065,0.030477,4.149149e-05,2.338520,0.082575,0.000024
2,0.000007,0.000003,0.000507,3.194201e-06,2.275094,0.176176,0.000018
3,0.011379,0.000180,0.743177,2.030291e-04,1.662012,0.034836,0.000008
4,0.321939,0.558842,0.010246,2.184037e-02,1.536406,0.001119,0.001203
...,...,...,...,...,...,...,...
1998,0.000002,0.000003,0.000372,1.483944e-06,2.436218,0.015190,0.000021
1999,0.001981,0.000856,0.058992,4.040857e-04,1.355186,1.033179,0.001024
2000,0.000109,0.001344,0.014503,3.742547e-03,2.431811,0.000114,0.000180
2001,0.000140,0.000237,0.068011,3.059243e-04,2.366347,0.016646,0.000047


In [57]:
(torch.tensor(ARes2.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8527209162712097